In [1]:
# Library requirements 
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam 
from tensorflow import keras
from tensorflow.keras.initializers import HeNormal, GlorotUniform, GlorotNormal
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors

2024-07-16 17:12:58.925172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## DML Models 
Selection of Differential Machine learning model architctures used for experimentation

In [5]:
class Autodiff(tf.keras.layers.Layer):
    """
    Custom Keras layer for automatic differentiation of a forward model's output with respect to its inputs.

    Computes the gradients of the forward model's predictions with respect to the input features,
    allowing for gradient-based optimization.

    Parameters:
    -----------
    fwd_model : keras.Model: forward model whose output gradients with respect to the inputs are to be computed.
    
    kwargs : dict: additional keyword arguments passed to the Keras Layer base class.

    Methods:
    --------
    call(input):
        Computes the gradient of the forward model's output with respect to the input features.
    """
    def __init__(self, fwd_model, **kwargs):
      super(Autodiff, self).__init__(**kwargs)
      self.units = None
      self.fwd_model = fwd_model 
    
    def call(self, input):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(input)
            pred_value = self.fwd_model(input)
           
        # Get the gradients dy/dx_i for each x feature x_i inputs 
        gradient = tape.gradient(pred_value, input)

        return gradient
    

In [ ]:

def build_model_dml(model_name, input_dim):
    """
    Builds and returns a multi-layer perceptron neural network model based on the specified model name.

    Parameters:
    -----------
    model_name : str: name of the model to build. ('model_dml', 'model_dml_BN', 'model_dml_big_BN')
    X_train : DataFrame: training data used to determine the input shape of the model.

    Returns:
    --------
    A Keras Sequential model configured based on the specified model name.
    """
    if model_name == 'model_dml':
        # feedforward
        input_1 = Input(shape=(input_dim,))
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(input_1)
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(x)
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(x)
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(x)
        y_pred = Dense(1, kernel_initializer='glorot_normal', activation='linear', name='y_pred')(x)

        fwd_model = tf.keras.models.Model(inputs=input_1, outputs=y_pred)

        # automatic differentiation
        autodiff_layer = Autodiff(fwd_model, name='dydx_pred')
        dydx_pred = autodiff_layer(input_1)

        model = tf.keras.models.Model(inputs=input_1, outputs=[y_pred, dydx_pred], name='Autodiff_Model')

        return model

    elif model_name == 'model_dml_BN':
        # feedforward
        input_1 = Input(shape=(input_dim,))
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(input_1)
        x = BatchNormalization()(x)
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(x)
        x = BatchNormalization()(x)
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(x)
        x = BatchNormalization()(x)
        x = Dense(20, kernel_initializer='glorot_normal', activation='softplus')(x)
        y_pred = Dense(1, kernel_initializer='glorot_normal', activation='linear', name='y_pred')(x)

        fwd_model = tf.keras.models.Model(inputs=input_1, outputs=y_pred)

        # automatic differentiation
        autodiff_layer = Autodiff(fwd_model, name='dydx_pred')
        dydx_pred = autodiff_layer(input_1)

        model = tf.keras.models.Model(inputs=input_1, outputs=[y_pred, dydx_pred], name='Autodiff_Model')

        return model

    elif model_name == 'model_dml_big_BN':
        # feedforward
        input_1 = Input(shape=(input_dim,))
        x = Dense(30, kernel_initializer='glorot_normal', activation='softplus')(input_1)
        x = BatchNormalization()(x)
        x = Dense(30, kernel_initializer='glorot_normal', activation='softplus')(x)
        x = BatchNormalization()(x)
        x = Dense(30, kernel_initializer='glorot_normal', activation='softplus')(x)
        x = BatchNormalization()(x)
        x = Dense(30, kernel_initializer='glorot_normal', activation='softplus')(x)
        y_pred = Dense(1, kernel_initializer='glorot_normal', activation='linear', name='y_pred')(x)

        fwd_model = tf.keras.models.Model(inputs=input_1, outputs=y_pred)

        # automatic differentiation
        autodiff_layer = Autodiff(fwd_model, name='dydx_pred')
        dydx_pred = autodiff_layer(input_1)

        model = tf.keras.models.Model(inputs=input_1, outputs=[y_pred, dydx_pred], name='Autodiff_Model')

        return model
    else:
        raise ValueError(f"Unsupported model name: {model_name}")


In [ ]:
class NormalisationLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer for normalizing inputs and outputs

    Normalizes the inputs and outputs of the model using mean and sd 
    Provides methods to scale and inverse scale both the outputs and the gradients.

    Attributes:
    -----------
    muX : array: Mean of the input features computed from the training data.
    stdX : array: Standard deviation of the input features computed from the training data.
    muY : array: Mean of the output values computed from the training data.
    stdY : array: Standard deviation of the output values computed from the training data.
    n : int: Number of input features.

    Methods:
    --------
    adapt(x_raw, y_raw, dydx_raw):
        Computes the mean and standard deviation for the inputs and outputs based on the training data.
    
    call(inputs):
        Normalizes the inputs based on the computed mean and standard deviation.
    
    yScaled(y):
        Scales the outputs based on the computed mean and standard deviation.
    
    yScaledInverse(y):
        Inverse scales the outputs based on the computed mean and standard deviation.
    
    dydxScaled(dydx):
        Scales the gradients based on the computed mean and standard deviation.
    
    dydxScaledInverse(dydx_scaled):
        Inverse scales the gradients based on the computed mean and standard deviation.
    
    output_n():
        Returns the number of input features.
    
    get_config():
        Returns the configuration of the layer for serialization.

    """
    def __init__(self, **kwargs):
        super(NormalisationLayer, self).__init__(**kwargs)

        self.muX = 0
        self.muY = 0 
        self.stdX = 0
        self.stdY = 0

        self.n = 0

    def adapt(self, x_raw, y_raw, dydx_raw):
        # basic processing (step 1 in the note)
        
        x0 = x_raw
        y0 = y_raw
        x0Bar = dydx_raw
        
        self.n = x_raw.shape[1] 
        m = x_raw.shape[0] # size of training set

        # normalize inputs
        # compute
        self.muX = x0.mean(axis=0)
        self.stdX = x0.std(axis=0)
       
        # normalize inputs
        # compute
        self.muY = y0.mean(axis=0)
        self.stdY = y0.std(axis=0)
        

    def call(self, inputs):
        # layer called on x as inputs
        return (inputs - self.muX) /  self.stdX
    
    def yScaled(self, y):
        return (y - self.muY) / self.stdY

    def yScaledInverse(self, y):
        return (y * self.stdY ) + self.muY

    def dydxScaled(self, dydx):
        return dydx * (self.stdX / self.stdY)
    
    def dydxScaledInverse(self, dydx_scaled):
        return dydx_scaled * (self.stdY / self.stdX)
    
    def output_n(self):
        return self.n

    def get_config(self):
        config = super(NormalisationLayer, self).get_config()
        return config
    


In [ ]:
class grad_custom_loss(keras.losses.Loss):
    """
    Custom loss function that incorporates gradient normalization
    Loss function normalizes the gradients of the predicted values during training

    Parameters:
    -----------
    norm_weights : array or None, optional: Weights for normalizing the gradients. If None, it will be computed from the training data.
    reduction : str, optional: Type of reduction to apply to loss. Default is `keras.losses.Reduction.AUTO`.
    name : str, optional: Name of the custom loss function  "grad_custom_loss".

    Methods:
    --------
    adapt(dydx_train):
        Adapts the normalization weights based on the gradients from the training data.
    
    call(y_true, y_pred):
        Computes the custom loss as the mean squared error between the true and predicted values, scaled by the normalization weights.

    """
    def __init__(self, norm_weights=None, reduction=keras.losses.Reduction.AUTO, name="grad_custom_loss"):
        super().__init__(reduction=reduction, name=name)
        self.norm_weights = norm_weights

    def adapt(self, dydx_train):
        arg = tf.convert_to_tensor(dydx_train, dtype=tf.float32)
        self.norm_weights = 1.0 / tf.reshape(tf.sqrt(tf.reduce_mean(arg ** 2, axis=0)), [1, -1])

    @tf.function
    def call(self, y_true, y_pred):
        return tf.math.reduce_mean(tf.square((y_true - y_pred) * self.norm_weights))


In [ ]:


def error_bars_first_order_estimate(df_train, df_test):
    """
    Computes first-order error bars for a given test set based on the nearest neighbors in the INDEX training set
    and the training sets ground truth and predicted graidents at the training points.

    Calculates various error metrics and first-order estimates for the market-to-market (MtM) values
    based on the spread and maturity duration, comparing the predicted MtM with the ground truth.

    Parameters:
    -----------
    df_train : DataFrame: training dataset containing the true and predicted values of cs01 and theta, along with other features.
    df_test : DataFrame: test dataset containing the true and predicted values of MtM, along with other features.

    Returns:
    --------
    results_df : DataFrame: containing the combined training and test datasets along with the calculated error metrics and first-order estimates.

    """

    # Calculate errors
    df_train['cs01_error'] = np.abs(df_train['cs01'] - df_train['cs01_pred'])
    df_train['theta_error'] = np.abs(df_train['theta'] - df_train['theta_pred'])

    train_matrix = df_train[['spread', 'mat_duration', 'coupon', 'recovery']].values
    test_matrix = df_test[['spread', 'mat_duration', 'coupon', 'recovery']].values

    # Find the nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=1).fit(train_matrix)
    distances, indices = nbrs.kneighbors(test_matrix)

    # For results 
    nearest_indices = indices.flatten()
    results_df = pd.concat([
        df_train.iloc[nearest_indices].reset_index(drop=True),
        df_test[['spread', 'mat_duration', 'coupon', 'recovery', 'mtm', 'mtm_pred']].reset_index(drop=True).rename(columns={
            'spread': 'spread_t',
            'mat_duration': 'mat_duration_t',
            'coupon': 'coupon_t',
            'recovery': 'recovery_t',
            'mtm': 'mtm_t',
            'mtm_pred': 'mtm_t_pred'
        })
    ], axis=1)

    ### New metrics to enable the error bar calculation ###
    results_df['ds'] = results_df['spread_t'] - results_df['spread']
    results_df['dt'] = results_df['mat_duration_t'] - results_df['mat_duration']

    # fo (ground truth) estimate of price
    results_df['mtm_gt_fo'] = results_df['mtm'] + (results_df['cs01'] * results_df['ds'] + results_df['theta'] * results_df['dt'])

    # foplus estimate of price
    results_df['mtm_gt_foplus'] = results_df['mtm'] + (
        (results_df['cs01'] + results_df['cs01_error']) * results_df['ds'] +
        (results_df['theta'] - results_df['theta_error']) * results_df['dt'])
    


    # Construct error bar
    results_df['error_bar_size'] = np.abs(results_df['mtm_gt_foplus'] - results_df['mtm_t_pred'])  + np.abs(results_df['mtm_t'] - results_df['mtm_gt_fo'])
    results_df['mtm_t_pred_up'] = results_df['mtm_t_pred'] + results_df['error_bar_size']
    results_df['mtm_t_pred_dn'] = results_df['mtm_t_pred'] - results_df['error_bar_size']
    results_df['error_bar_size_gt'] = np.abs(results_df['mtm_gt_fo'] - results_df['mtm_t'])
    results_df['mtm_t_up'] = results_df['mtm_t'] + results_df['error_bar_size_gt']
    results_df['mtm_t_dn'] = results_df['mtm_t'] - results_df['error_bar_size_gt']

    ### Define some error metrics for info only ###
    results_df['gt_nn_error'] = (results_df['mtm_t'] - results_df['mtm_t_pred']) / results_df['mtm_t'] * 100
    results_df['gt_fo_error'] = (results_df['mtm_gt_fo'] - results_df['mtm_t']) / results_df['mtm_t'] * 100
    results_df['nn_fo_error'] = (results_df['mtm_gt_fo'] - results_df['mtm_t_pred']) / results_df['mtm_t_pred'] * 100

    # Check if mtm_t prediction falls within the error bars
    results_df['mtm_t_pred_within_nn_error'] = (results_df['mtm_t_pred'] <= results_df['mtm_t_pred_up']) & (results_df['mtm_t_pred'] >= results_df['mtm_t_pred_dn'])
    results_df['mtm_t_within_nn_error'] = (results_df['mtm_t'] <= results_df['mtm_t_pred_up']) & (results_df['mtm_t'] >= results_df['mtm_t_pred_dn'])
    results_df['mtm_t_pred_within_gt_error'] = (results_df['mtm_t_pred'] <= results_df['mtm_t_up']) & (results_df['mtm_t_pred'] >= results_df['mtm_t_dn'])
    results_df['mtm_t_within_gt_error'] = (results_df['mtm_t'] <= results_df['mtm_t_up']) & (results_df['mtm_t'] >= results_df['mtm_t_dn'])

    # How close to the error bar are the points that are not within the error bar
    results_df.loc[~results_df['mtm_t_within_nn_error'], 'true_misses'] = np.minimum(
        np.abs(results_df['mtm_t'] - results_df['mtm_t_pred_up']),
        np.abs(results_df['mtm_t'] - results_df['mtm_t_pred_dn'])
    ) / (2 * results_df['error_bar_size']) * 100

    # Size of error bars as a percentage of the prediction price
    results_df['error_bar_percent_of_price'] = (2 * results_df['error_bar_size']) / np.abs(results_df['mtm_t_pred']) * 100
    results_df['rel_size'] = np.abs(results_df['mtm_t_pred_up'] - results_df['mtm_t_pred_dn']) / np.abs(results_df['mtm_t_up'] - results_df['mtm_t_dn'])

    # Return the results data frame
    return results_df


In [ ]:


def error_bars_first_order_estimate_opt(df_train, df_test):
    """
    Computes first-order error bars for a given test set based on the nearest neighbors in the OPTION training set
    and the training sets ground truth and predicted graidents at the training points.

    Calculates various error metrics and first-order estimates for the market-to-market (MtM) values
    based on the spread and expiry duration, strike and implied volatility; comparing the predicted MtM with the ground truth.

    Parameters:
    -----------
    df_train : DataFrame: training dataset containing the true and predicted values of cs01 and theta, along with other features.
    df_test : DataFrame: test dataset containing the true and predicted values of MtM, along with other features.

    Returns:
    --------
    results_df : DataFrame: containing the combined training and test datasets along with the calculated error metrics and first-order estimates.

    """
  
    # Calculate errors
    df_train['delta_error'] = np.abs(df_train['delta'] - df_train['delta_pred'])
    df_train['theta_error'] = np.abs(df_train['theta'] - df_train['theta_pred'])
    df_train['deltastrike_error'] = np.abs(df_train['deltastrike'] - df_train['deltastrike_pred'])
    df_train['vega_error'] = np.abs(df_train['vega'] - df_train['vega_pred'])
  
    train_matrix = df_train[['spread', 'exp_duration', 'strike', 'impliedVol']].values
    test_matrix = df_test[['spread', 'exp_duration', 'strike', 'impliedVol']].values
  
    # Find the nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=1).fit(train_matrix)
    distances, indices = nbrs.kneighbors(test_matrix)
  
    # Extract the indices of the nearest neighbors
    nearest_indices = indices.flatten()
  
    # For the results
    results_df = pd.concat([
        df_train.iloc[nearest_indices].reset_index(drop=True),
        df_test[['spread', 'exp_duration', 'mat_duration', 'strike', 'impliedVol', 'mtm', 'mtm_pred']].reset_index(drop=True).rename(columns={
            'spread': 'spread_t',
            'exp_duration': 'exp_duration_t',
            'mat_duration': 'mat_duration_t',
            'strike': 'strike_t',
            'impliedVol': 'impliedVol_t',
            'mtm': 'mtm_t',
            'mtm_pred': 'mtm_t_pred'
        })
    ], axis=1)
  
    ### New metrics to enable the error bar calculation ###
    results_df['ds'] = results_df['spread_t'] - results_df['spread']
    results_df['dt'] = results_df['exp_duration_t'] - results_df['exp_duration']
    results_df['dk'] = results_df['strike_t'] - results_df['strike']
    results_df['dv'] = results_df['impliedVol_t'] - results_df['impliedVol']
  
    # fo (ground truth) estimate of price
    results_df['mtm_gt_fo'] = results_df['mtm'] + (results_df['delta'] * results_df['ds'] + results_df['theta'] * results_df['dt'] + results_df['deltastrike'] * results_df['dk'] + results_df['vega'] * results_df['dv'])
  
    # foplus estimate of price
    results_df['mtm_gt_foplus'] = results_df['mtm'] + (
        (results_df['delta'] + results_df['delta_error']) * results_df['ds'] +
        (results_df['theta'] + results_df['theta_error']) * results_df['dt'] +
        (results_df['deltastrike'] + results_df['deltastrike_error']) * results_df['dk'] +
        (results_df['vega'] + results_df['vega_error']) * results_df['dv']
    )
  
    # Construct error bar
    results_df['error_bar_size'] = np.abs(results_df['mtm_gt_foplus'] - results_df['mtm_t_pred'])  + np.abs(results_df['mtm_t'] - results_df['mtm_gt_fo'])
    results_df['mtm_t_pred_up'] = results_df['mtm_t_pred'] + results_df['error_bar_size']
    results_df['mtm_t_pred_dn'] = results_df['mtm_t_pred'] - results_df['error_bar_size']
    results_df['error_bar_size_gt'] = np.abs(results_df['mtm_gt_fo'] - results_df['mtm_t'])
    results_df['mtm_t_up'] = results_df['mtm_t'] + results_df['error_bar_size_gt']
    results_df['mtm_t_dn'] = results_df['mtm_t'] - results_df['error_bar_size_gt']
  
    ### Define the error metrics ###
    # The 'real error' in prediction between the real gt price and the nn prediction
    results_df['gt_nn_error'] = (results_df['mtm_t'] - results_df['mtm_t_pred']) / results_df['mtm_t'] * 100
    # Error between gt price and the gt fo estimate of the price
    results_df['gt_fo_error'] = (results_df['mtm_gt_fo'] - results_df['mtm_t']) / results_df['mtm_t'] * 100
    # Error between nn prediction and gt fo estimate (gt fo estimate could be used as a floor/cap of mtm_t_pred)
    results_df['nn_fo_error'] = (results_df['mtm_gt_fo'] - results_df['mtm_t_pred']) / results_df['mtm_t_pred'] * 100
  
    # Remove rows where ds = dt = dk = dv = 0
    #results_df = results_df[~((results_df['ds'] == 0) & (results_df['dt'] == 0) & (results_df['dk'] == 0) & (results_df['dv'] == 0))]
  
    # Check if mtm_t prediction falls within the error bars
    results_df['mtm_t_pred_within_nn_error'] = (results_df['mtm_t_pred'] <= results_df['mtm_t_pred_up']) & (results_df['mtm_t_pred'] >= results_df['mtm_t_pred_dn'])
    results_df['mtm_t_within_nn_error'] = (results_df['mtm_t'] <= results_df['mtm_t_pred_up']) & (results_df['mtm_t'] >= results_df['mtm_t_pred_dn'])
    results_df['mtm_t_pred_within_gt_error'] = (results_df['mtm_t_pred'] <= results_df['mtm_t_up']) & (results_df['mtm_t_pred'] >= results_df['mtm_t_dn'])
    results_df['mtm_t_within_gt_error'] = (results_df['mtm_t'] <= results_df['mtm_t_up']) & (results_df['mtm_t'] >= results_df['mtm_t_dn'])
  
    # How close to the error bar are the points that are not within the error bar
    results_df.loc[~results_df['mtm_t_within_nn_error'], 'true_misses'] = np.minimum(
        np.abs(results_df['mtm_t'] - results_df['mtm_t_pred_up']),
        np.abs(results_df['mtm_t'] - results_df['mtm_t_pred_dn'])
    ) / (2 * results_df['error_bar_size']) * 100
  
    # Size of error bars as a percentage of the prediction price
    results_df['error_bar_percent_of_price'] = (2 * results_df['error_bar_size']) / np.abs(results_df['mtm_t_pred']) * 100
    results_df['rel_size'] = np.abs(results_df['mtm_t_pred_up'] - results_df['mtm_t_pred_dn']) / np.abs(results_df['mtm_t_up'] - results_df['mtm_t_dn'])
  
    # Return the results data frame
    return results_df


## Common Data Processing Functions

In [ ]:
def preprocess_data(x_train, y_train, dydx_train, prep_type='Normalisation'):
    """
    Preprocesses the training data. Only one preparation type currently. 

    Normalizes the input features and gradients, and initializes a custom gradient-based loss function
    Other preparation types can be added if required

    Parameters:
    -----------
    x_train : array or DataFrame: training input features.
    y_train : array or DataFrame: training target values.
    dydx_train : array or DataFrame: gradients of the training data.
    prep_type : str, optional: type of data preparation to perform. ()'Normalisation')

    Returns:
    --------
    prep_layer : NormalisationLayer: normalization layer initialized and adapted to the training data.
    GradLoss : grad_custom_loss: custom gradient-based loss function adapted to the normalized gradients.

    """

    if prep_type == 'Normalisation':
        prep_layer = NormalisationLayer(input_shape=[x_train.shape[1],])    
        prep_layer.adapt(x_train, y_train, dydx_train)

        GradLoss = grad_custom_loss()
        GradLoss.adapt(prep_layer(dydx_train))
    
        return prep_layer, GradLoss
    
    # Add other scaling if need be here ...
    else:
         raise ValueError(f"Unsupported data preparation type: {prep_type}")
    

    
# predict and inverse data transform   
def predict_unscaled(model, prep_layer, x_unscaled):
    """
    Makes predictions using a trained model on unscaled input data and returns the unscaled predictions and gradients.

    First scales the input data using the provided preprocessing layer, makes predictions using the
    provided model, and then inversely scales the predictions and gradients to return them in their original scale.

    Parameters:
    -----------
    model : keras.Model: trained Keras model used for making predictions.
    prep_layer : NormalisationLayer: normalization layer used to scale and inverse scale the input data and predictions.
    x_unscaled : array or DataFrame: unscaled input data for which predictions are to be made.

    Returns:
    --------
    y_pred : array: unscaled predicted values.
    dydx_pred : array: unscaled predicted gradients.
    """

    y_scaled, dydx_scaled = model.predict(prep_layer(x_unscaled))
    y_pred = prep_layer.yScaledInverse(y_scaled)
    dydx_pred = prep_layer.dydxScaledInverse(dydx_scaled)
    
    return y_pred.reshape(-1,1), dydx_pred

## Common Build, Compile and Fit Functions

In [ ]:
def build_and_compile_model(
        input_dim,
        model_name,
        grad_loss,
        differential_weight,
        lr_schedule
    ):
    """
    Builds and compiles a model with the specified parameters, including a custom gradient-based loss function and learning rate schedule.

    Parameters:
    -----------
    input_dim : int: number of input features for the model.
    model_name : str: name of the model. 
    grad_loss : grad_custom_loss: custom gradient-based loss function to use for training.
    differential_weight : float:  weight factor for balancing the main loss and gradient loss.
    lr_schedule : tf.keras.optimizers.schedules.LearningRateSchedule: learning rate schedule to use with the Adam optimizer.

    Returns:
    --------
    model : keras.Model:  compiled Keras model
    """

    model = build_model_dml(model_name, input_dim)
    alpha = 1.0 / (1.0 + differential_weight * input_dim)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss={ 'y_pred': 'mse', 'dydx_pred' : grad_loss},
        run_eagerly=None,
        loss_weights=[alpha,1-alpha]
    )

    return model


def train_model(model,
                prep_layer, 
                x_train, 
                y_train, 
                dydx_train=None,
                epochs = config['epochs'],
                batch_size = 1024,
                x_true = None,
                y_true = None,
                dydx_true = None):
    
    """
    Trains a given model with the provided training data and preprocessing layer.

    Parameters:
    -----------
    model : keras.Model: model to be trained.
    prep_layer : NormalisationLayer: normalization layer used to scale the input and output data.
    x_train : array or DataFrame: unscaled training input features.
    y_train : array or DataFrame: unscaled training target values.
    dydx_train : array or DataFrame, optional: unscaled training gradients. 
    epochs : int, optional: number of epochs to train the model.
    batch_size : int, optional: batch size 
    x_true : array or DataFrame, optional: unscaled validation input features.
    y_true : array or DataFrame, optional:  unscaled validation target values. 
    dydx_true : array or DataFrame, optional: unscaled validation gradients.

    Returns:
    --------
    history : keras.callbacks.History: record of training loss values and metrics for training and validation data.  
    """

    history = model.fit(
        prep_layer(x_train), [prep_layer.yScaled(y_train), prep_layer.dydxScaled(dydx_train)], 
        batch_size = batch_size,
        epochs=epochs,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=100)],
        validation_data = (prep_layer(x_true), [prep_layer.yScaled(y_true), prep_layer.dydxScaled(dydx_true)]),
        verbose=1
        )
    return history